# Getting Started with Google Search as a Tool with Gemini in Vertex AI

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb

### Set up the notebook

In [ ]:
# 1. Install Google Gen AI SDK for Python
# Install the following packages required to execute this notebook.

%pip install --upgrade --quiet google-genai

# Restart runtime
# To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

import IPython
​
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# -->⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️


# 2. Authenticate your Google Cloud account
# If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

import sys
​
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth
​
    auth.authenticate_user()